<a href="https://colab.research.google.com/github/martine-augustin/Marionnaud/blob/main/Marionnaud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marionnaud projet


## Variable et installation

In [15]:
# Install
!pip install beautifulsoup4
!pip install lxml




In [16]:
# Import
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


## Scrapping

In [17]:
# 1. L'URL de la page à scraper
url = 'https://fr.trustpilot.com/review/www.marionnaud.com'

# 2. Récupérer le contenu de la page avec 'requests'
response = requests.get(url)

# Vérifier si la requête a réussi (Code 200)
if response.status_code == 200:

    # 3. Créer l'objet BeautifulSoup pour parser le HTML
    # 'html.parser' est le parser inclus dans Python
    soup = BeautifulSoup(response.text, 'html.parser')

    # --- EXEMPLE 1 : Récupérer le titre de la page ---
    titre = soup.title.string
    print(f"Titre de la page : {titre}\n")

    # --- EXEMPLE 2 : Trouver un élément spécifique (le premier h1) ---
    h1 = soup.find('h1')
    print(f"Titre H1 : {h1.text}")

    # --- EXEMPLE 3 : Trouver TOUS les liens (balises <a>) ---
    print("\n--- Liste des liens ---")
    liens = soup.find_all('a') # Trouve toutes les balises <a>

    for lien in liens:
        # On récupère le texte du lien et son URL (href)
        texte_du_lien = lien.text
        url_du_lien = lien.get('href')
        print(f"Texte : {texte_du_lien} -> Lien : {url_du_lien}")

else:
    print("Erreur lors de la récupération de la page.")

Titre de la page : Avis de Marionnaud France | Lisez les avis marchands de www.marionnaud.com

Titre H1 : Marionnaud France Avis 7 307

--- Liste des liens ---
Texte :  -> Lien : /
Texte : Marionnauditaliamarionnaud.it•780 avis1.6 -> Lien : /review/marionnaud.it
Texte : Marionnaudmarionnaud.fr•176 avis1.7 -> Lien : /review/marionnaud.fr
Texte : Sephorawww.sephora.fr•3,2 k avis2.2 -> Lien : /review/www.sephora.fr
Texte : Catégories -> Lien : /categories
Texte : Blog -> Lien : /blog
Texte : Connexion -> Lien : /users/connect?redirect=&source_cta=header
Texte : Pour les entreprises -> Lien : https://fr.business.trustpilot.com
Texte : Pour les entreprises -> Lien : https://fr.business.trustpilot.com
Texte : Connexion -> Lien : /users/connect?redirect=&source_cta=header
Texte : Catégories -> Lien : /categories
Texte : Blog -> Lien : /blog
Texte : Beauté & bien-être -> Lien : /categories/beauty_wellbeing
Texte : Produits de beauté & maquillage -> Lien : /categories/cosmetics_makeup
Texte : F